# Sentiment Analysis (SA) with pretrained Language Model (LM)

In this notebook, we are going to build a sentiment analysis model based on the pretrained
language model. We are focusing on the best usability to support traditional nlp tasks in a simple fashion. The building process is really simple in three steps. Let us get started now.

## Preparation and settings

### Load mxnet and gluonnlp

In [1]:
import time
import multiprocessing as mp
import numpy as np

import mxnet as mx
from mxnet import nd, gluon, autograd

import gluonnlp as nlp
from gluonnlp import data, Vocab

### Hyperparameters

In [2]:
dropout = 0
language_model_name = 'standard_lstm_lm_200'
pretrained = True
num_gpus = 1
learning_rate = 0.005 * num_gpus
batch_size = 16 * num_gpus
bucket_num = 10
bucket_ratio = 0.5
epochs = 1
grad_clip = 2
log_interval = 100

In [3]:
context = [mx.gpu(i) for i in range(num_gpus)] if num_gpus else [mx.cpu()]

## Sentiment analysis model with pre-trained language model encoder

The model architecture is based on pretrained LM:

![sa-model](samodel-v3.png)

In [4]:
class SentimentNet(gluon.Block):
    def __init__(self, embedding_block, encoder_block, dropout, prefix=None, params=None):
        super(SentimentNet, self).__init__(prefix=prefix, params=params)
        with self.name_scope():
            self.embedding = embedding_block
            self.encoder = encoder_block
            self.out_layer = gluon.nn.HybridSequential()
            with self.out_layer.name_scope():
                self.out_layer.add(gluon.nn.Dropout(dropout))
                self.out_layer.add(gluon.nn.Dense(1, flatten=False))

    def forward(self, data, valid_length):
        encoded = self.encoder(nd.Dropout(self.embedding(data), 0.6, axes=(0,)))  # Shape(T, N, C)
        masked_encoded = nd.SequenceMask(encoded,
                                        sequence_length=valid_length,
                                        use_sequence_length=True)
        agg_state = nd.broadcast_div(nd.sum(masked_encoded, axis=0),
                                     nd.expand_dims(valid_length, axis=1))
        out = self.out_layer(agg_state)
        return out

lm_model, vocab = nlp.model.get_model(name=language_model_name,
                                      pretrained=pretrained,
                                      ctx=context,
                                      dropout=dropout,
                                      prefix='sent_net_')
net = SentimentNet(embedding_block=lm_model.embedding, encoder_block=lm_model.encoder,
                   dropout=dropout, prefix='sent_net_')
net.initialize(mx.init.Xavier(), ctx=context)
net.hybridize()
print(net)

SentimentNet(
  (out_layer): HybridSequential(
    (0): Dropout(p = 0, axes=())
    (1): Dense(None -> 1, linear)
  )
  (embedding): HybridSequential(
    (0): Embedding(33278 -> 200, float32)
  )
  (encoder): LSTM(200 -> 800, TNC, num_layers=2)
)


/usr/local/lib/python3.5/dist-packages/mxnet/gluon/parameter.py:321: UserWarning: Parameter sent_net_hybridsequential0_embedding0_weight is already initialized, ignoring. Set force_reinit=True to re-initialize.
  "Set force_reinit=True to re-initialize."%self.name)
/usr/local/lib/python3.5/dist-packages/mxnet/gluon/parameter.py:321: UserWarning: Parameter sent_net_lstm0_l0_i2h_weight is already initialized, ignoring. Set force_reinit=True to re-initialize.
  "Set force_reinit=True to re-initialize."%self.name)
/usr/local/lib/python3.5/dist-packages/mxnet/gluon/parameter.py:321: UserWarning: Parameter sent_net_lstm0_l0_h2h_weight is already initialized, ignoring. Set force_reinit=True to re-initialize.
  "Set force_reinit=True to re-initialize."%self.name)
/usr/local/lib/python3.5/dist-packages/mxnet/gluon/parameter.py:321: UserWarning: Parameter sent_net_lstm0_l0_i2h_bias is already initialized, ignoring. Set force_reinit=True to re-initialize.
  "Set force_reinit=True to re-initialize

## Data pipeline

### Load sentiment analysis dataset -- IMDB reviews

In [5]:
train_dataset, test_dataset = [data.IMDB(root='data/imdb', segment=segment) for segment in ('train', 'test')]
print("Tokenize using spaCy...")
tokenizer = nlp.data.SpacyTokenizer('en')
length_clip = nlp.data.ClipSequence(500)

def preprocess(x):
    data, label = x
    label = int(label > 5)
    data = vocab[length_clip(tokenizer(data))]
    return data, label, float(len(data))

def get_length(x):
    return x[2]

def preprocess_dataset(dataset):
    start = time.time()
    with mp.Pool(32) as pool:
        dataset = gluon.data.SimpleDataset(pool.map(preprocess, dataset))
        lengths = gluon.data.SimpleDataset(pool.map(get_length, dataset))
    end = time.time()
    print('Done! Tokenizing Time={:.2f}s, #Sentences={}'.format(end - start, len(dataset)))
    return dataset, lengths

train_dataset, train_data_lengths = preprocess_dataset(train_dataset)
test_dataset, test_data_lengths = preprocess_dataset(test_dataset)

Tokenize using spaCy...
Done! Tokenizing Time=5.26s, #Sentences=25000
Done! Tokenizing Time=4.97s, #Sentences=25000



## Training

### Evaluation using loss and accuracy

In [6]:
def evaluate(net, dataloader, context):
    loss = gluon.loss.SigmoidBCELoss()
    total_L = 0.0
    total_sample_num = 0
    total_correct_num = 0
    start_log_interval_time = time.time()
    print('Begin Testing...')
    for i, (data, label, valid_length) in enumerate(dataloader):
        data = mx.nd.transpose(data.as_in_context(context))
        valid_length = valid_length.as_in_context(context).astype(np.float32)
        label = label.as_in_context(context)
        output = net(data, valid_length)
        L = loss(output, label)
        pred = (output > 0.5).reshape(-1)
        total_L += L.sum().asscalar()
        total_sample_num += label.shape[0]
        total_correct_num += (pred == label).sum().asscalar()
        if (i + 1) % log_interval == 0:
            print('[Batch {}/{}] elapsed {:.2f} s'.format(
                i + 1, len(dataloader), time.time() - start_log_interval_time))
            start_log_interval_time = time.time()
    avg_L = total_L / float(total_sample_num)
    acc = total_correct_num / float(total_sample_num)
    return avg_L, acc

In [7]:
def train(net, context, epochs):
    trainer = gluon.Trainer(net.collect_params(), 'ftml', {'learning_rate': learning_rate})
    loss = gluon.loss.SigmoidBCELoss()

    # Construct the DataLoader
    batchify_fn = nlp.data.batchify.Tuple(nlp.data.batchify.Pad(axis=0), 
                                          nlp.data.batchify.Stack(),
                                          nlp.data.batchify.Stack()) # Pad data, stack label and lengths
    batch_sampler = nlp.data.sampler.FixedBucketSampler(train_data_lengths,
                                                        batch_size=batch_size,
                                                        num_buckets=bucket_num,
                                                        ratio=bucket_ratio,
                                                        shuffle=True)
    print(batch_sampler.stats())
    train_dataloader = gluon.data.DataLoader(dataset=train_dataset,
                                             batch_sampler=batch_sampler,
                                             batchify_fn=batchify_fn)
    test_dataloader = gluon.data.DataLoader(dataset=test_dataset,
                                            batch_size=batch_size,
                                            shuffle=False,
                                            batchify_fn=batchify_fn)
    parameters = net.collect_params().values()

    # Training/Testing
    for epoch in range(epochs):
        # Epoch training stats
        start_epoch_time = time.time()
        epoch_L = 0.0
        epoch_sent_num = 0
        epoch_wc = 0
        # Log interval training stats
        start_log_interval_time = time.time()
        log_interval_wc = 0
        log_interval_sent_num = 0
        log_interval_L = 0.0

        for i, (data, label, length) in enumerate(train_dataloader):
            if data.shape[0] > len(context):
                data_list = gluon.utils.split_and_load(data, context, batch_axis=0, even_split=False)
                label_list = gluon.utils.split_and_load(label, context, batch_axis=0, even_split=False)
                length_list = gluon.utils.split_and_load(length, context, batch_axis=0, even_split=False)
            else:
                data_list = [data.as_in_context(context[0])]
                label_list = [label.as_in_context(context[0])]
                length_list = [length.as_in_context(context[0])]
            L = 0
            wc = length.sum().asscalar()
            log_interval_wc += wc
            epoch_wc += wc
            log_interval_sent_num += data.shape[1]
            epoch_sent_num += data.shape[1]
            for data, label, valid_length in zip(data_list, label_list, length_list):
                valid_length = valid_length
                with autograd.record():
                    output = net(data.T, valid_length)
                    L = L + loss(output, label).mean().as_in_context(context[0])
            L.backward()
            # Clip gradient
            if grad_clip:
                gluon.utils.clip_global_norm([p.grad(x.context) for p in parameters for x in data_list],
                                             grad_clip)
            # Update parameter
            trainer.step(1)
            log_interval_L += L.asscalar()
            epoch_L += L.asscalar()
            if (i + 1) % log_interval == 0:
                print('[Epoch {} Batch {}/{}] elapsed {:.2f} s, avg loss {:.6f}, throughput {:.2f}K wps'.format(
                    epoch, i + 1, len(train_dataloader), time.time() - start_log_interval_time,
                    log_interval_L / log_interval_sent_num,
                    log_interval_wc / 1000 / (time.time() - start_log_interval_time)))
                # Clear log interval training stats
                start_log_interval_time = time.time()
                log_interval_wc = 0
                log_interval_sent_num = 0
                log_interval_L = 0
        end_epoch_time = time.time()
        test_avg_L, test_acc = evaluate(net, test_dataloader, context[0])
        print('[Epoch {}] train avg loss {:.6f}, test acc {:.2f}, test avg loss {:.6f}, throughput {:.2f}K wps'.format(
            epoch, epoch_L / epoch_sent_num,
            test_acc, test_avg_L, epoch_wc / 1000 / (end_epoch_time - start_epoch_time)))

In [8]:
train(net, context, epochs)

FixedBucketSampler:
  sample_num=25000, batch_num=1319
  key=[14, 68, 122, 176, 230, 284, 338, 392, 446, 500]
  cnt=[5, 976, 2353, 6662, 4470, 2661, 1836, 1385, 1012, 3640]
  batch_size=[285, 58, 32, 22, 17, 16, 16, 16, 16, 16]
[Epoch 0 Batch 100/1319] elapsed 9.39 s, avg loss 0.002415, throughput 50.44K wps
[Epoch 0 Batch 200/1319] elapsed 8.70 s, avg loss 0.002434, throughput 52.44K wps
[Epoch 0 Batch 300/1319] elapsed 8.00 s, avg loss 0.002218, throughput 61.50K wps
[Epoch 0 Batch 400/1319] elapsed 8.24 s, avg loss 0.002379, throughput 56.03K wps
[Epoch 0 Batch 500/1319] elapsed 7.40 s, avg loss 0.002367, throughput 61.94K wps
[Epoch 0 Batch 600/1319] elapsed 8.13 s, avg loss 0.002169, throughput 59.47K wps
[Epoch 0 Batch 700/1319] elapsed 8.25 s, avg loss 0.002181, throughput 58.41K wps
[Epoch 0 Batch 800/1319] elapsed 8.37 s, avg loss 0.001969, throughput 58.19K wps
[Epoch 0 Batch 900/1319] elapsed 8.83 s, avg loss 0.002150, throughput 50.71K wps
[Epoch 0 Batch 1000/1319] elapsed 

In [9]:
net(mx.nd.reshape(mx.nd.array(vocab[['This', 'movie', 'is', 'amazing']], ctx=context[0]), shape=(-1, 1)),
    mx.nd.array([4], ctx=context[0])).sigmoid()


[[0.7000658]]
<NDArray 1x1 @gpu(0)>

## Conclusion

In summary, we have built a SA model using gluonnlp. It is:

1) easy to use.

2) simple to customize.

3) fast to build the NLP prototype.

Gluonnlp documentation is here: http://gluon-nlp.s3-accelerate.dualstack.amazonaws.com/index.html